In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from PIL import Image
from io import BytesIO
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from google.colab.patches import cv2_imshow
from transformers import pipeline
import requests

## Section 1: Language Processing
Application: Sentiment Analysis

In [ ]:
# Load a pre-trained sentiment analysis pipeline from Hugging Face
sentiment_analysis = pipeline("sentiment-analysis")

In [ ]:
# Example text data
texts = [
    "I love sunny days!",
    "I hate when it rains all day.",
    "The weather is just perfect.",
    "I'm feeling quite sad today."
]

In [ ]:
# Perform sentiment analysis on each text
results = sentiment_analysis(texts)

In [ ]:
# Display the texts and their sentiment results
for text, result in zip(texts, results):
    print(f"Text: {text}\nSentiment: {result['label']} (Score: {result['score']})\n")

## Section 2: Computer Vision
Application: Object Detection

In [ ]:
# Function to download and display an image
def display_image(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content)).convert("RGB")
    plt.imshow(img)
    plt.axis('off')
    plt.show()
    return img

In [ ]:
# Load a pre-trained object detection model from TensorFlow Hub
detector = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")

In [ ]:
# Example image URL
image_url = "https://images.unsplash.com/photo-1593642532973-d31b6557fa68"

In [ ]:
# Display the image
img = display_image(image_url)

In [ ]:
# Convert the image to a tensor
img = img.resize((640, 480))  # Resize to match model's expected input size
img_tensor = tf.convert_to_tensor(np.array(img), dtype=tf.float32)
img_tensor = img_tensor / 255.0  # Normalize to [0, 1]
img_tensor = tf.expand_dims(img_tensor, axis=0)  # Add batch dimension

In [ ]:
# Run the object detection model
result = detector(img_tensor)

In [ ]:
# Function to draw bounding boxes on the image
def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.5):
    colors = list(plt.cm.tab20(np.linspace(0, 1, 20)))
    font = cv2.FONT_HERSHEY_SIMPLEX
    for i in range(min(max_boxes, boxes.shape[0])):
        if scores[i] >= min_score:
            color = colors[int(class_names[i]) % len(colors)]
            ymin, xmin, ymax, xmax = tuple(boxes[i])
            image = cv2.rectangle(image, (int(xmin * img.width), int(ymin * img.height)),
                                  (int(xmax * img.width), int(ymax * img.height)), color, 2)
            image = cv2.putText(image, str(int(class_names[i])), (int(xmin * img.width), int(ymin * img.height) - 10),
                                font, 0.5, color, 2)
    return image

In [ ]:
# Extract detection results
boxes = result["detection_boxes"][0].numpy()
class_names = result["detection_classes"][0].numpy()
scores = result["detection_scores"][0].numpy()

In [ ]:
# Draw bounding boxes on the image
output_img = draw_boxes(np.array(img), boxes, class_names, scores)

In [ ]:
# Display the image with bounding boxes
cv2_imshow(output_img)

## Section 3: Natural Language Processing
Application: Text Similarity

In [ ]:
# Example text data
text1 = "AI is transforming the world."
text2 = "Artificial intelligence is changing everything."

In [ ]:
# Convert text data to embeddings
embedding1 = sentiment_model([text1])[0]
embedding2 = sentiment_model([text2])[0]

In [ ]:
# Calculate cosine similarity between the two texts
similarity = cosine_similarity([embedding1], [embedding2])[0][0]

In [ ]:
# Display the texts and their similarity score
print(f"Text 1: {text1}\nText 2: {text2}\nSimilarity Score: {similarity}")